In [1]:
import os
import coinmarketcap
import numpy as np
import pandas as pd
import json
import numpy
import sentiment_analysis as sa
import refreshTable
from flask import *

In [50]:
#coin = market.ticker(limit = 200)
#print(type(coin))
#print(dir(coin))
print(coin)

[{'cached': False, 'available_supply': '16988075.0', 'name': 'Bitcoin', 'max_supply': '21000000.0', 'symbol': 'BTC', 'price_btc': '1.0', 'rank': '1', 'percent_change_24h': '2.42', 'percent_change_1h': '0.42', 'id': 'bitcoin', '24h_volume_usd': '7309420000.0', 'market_cap_usd': '142329999607', 'percent_change_7d': '7.16', 'total_supply': '16988075.0', 'price_usd': '8378.23', 'last_updated': '1524190774'}, {'cached': False, 'available_supply': '98927557.0', 'name': 'Ethereum', 'max_supply': None, 'symbol': 'ETH', 'price_btc': '0.0699673', 'rank': '2', 'percent_change_24h': '10.1', 'percent_change_1h': '1.15', 'id': 'ethereum', '24h_volume_usd': '2423380000.0', 'market_cap_usd': '57731154693.0', 'percent_change_7d': '19.97', 'total_supply': '98927557.0', 'price_usd': '583.57', 'last_updated': '1524190759'}, {'cached': False, 'available_supply': '39122794968.0', 'name': 'Ripple', 'max_supply': '100000000000', 'symbol': 'XRP', 'price_btc': '0.00010328', 'rank': '3', 'percent_change_24h': '1

In [18]:
market = coinmarketcap.Market()
coins = (market.ticker())
ether = pd.DataFrame(market.ticker(limit = 180)).set_index("id")
ether['percent_change_1h'] = ether['percent_change_1h'].astype(float)
#print(ether['percent_change_1h'] + '% $' + ether['symbol'])
#print(ether.loc['bitcoin'])
ethsort = ether.sort_values('percent_change_1h', axis = 'index', ascending = 0)
ethsort['symbol'] = '$' + ethsort['symbol']
a = ['symbol','percent_change_1h','percent_change_24h','percent_change_7d','price_btc',
     'price_usd','market_cap_usd','24h_volume_usd']
ethsort = ethsort[a]
#display(ethsort)

In [19]:
extractor = sa.twitter_setup()
counter = 0
for symb in ethsort['symbol']:
    counter = counter + 1
    # tweet list
    try:
        tweets = extractor.search(symb, count = 10)
    except sa.tweepy.TweepError:
        break
    # create a pandas dataframe as follows:
    tweetData = pd.DataFrame(data=[tweet.created_at for tweet in tweets], columns=['Date'])
    # add relevant data:
    tweetData['Tweets'] = np.array([tweet.text for tweet in tweets])
    tweetData['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
    tweetData['RTs']    = np.array([tweet.retweet_count for tweet in tweets])
    # create a column with the result of the analysis:
    tweetData['SA'] = np.array([ sa.analyze_sentiment(tweet) for tweet in tweetData['Tweets'] ])
    # We display the updated dataframe with the new column:
    print(symb)
    print(counter)
    #display(tweetData.head(10))
    #print(tweetData.head(10))
#print(sa.tweepy.TweepError)

$DCN
1
$BCO
2
$PPP
3
$NAS
4
$LOOM
5
$SPHTX
6
$TKY
7
$MAN
8
$EMC
9
$SMART
10
$SAN
11
$OMG
12
$GAS
13
$AGI
14
$GTO
15
$POA
16
$BOS
17
$WAX
18
$ZIL
19
$KIN
20
$PAYX
21
$WAVES
22
$ETH
23
$GNT
24
$SRN
25
$CVC
26
$MANA
27
$CMT
28
$MKR
29
$ZRX
30
$POWR
31
$ANT
32
$STEEM
33
$WICC
34
$XRP
35
$CND
36
$KMD
37
$POE
38
$LTC
39
$IOST
40
$ETHOS
41
$BNT
42
$GXS
43
$MAID
44
$HPB
45
$POLY
46
$STORM
47
$ABT
48
$BCH
49
$AE
50
$XEM
51
$XVG
52
$BTG
53
$RDN
54
$AMB
55
$DGB
56
$TOMO
57
$ARK
58
$MITH
59
$SUB
60
$ICN
61
$ONT
62
$BFT
63
$BNB
64
$WTC
65
$DGD
66
$GAME
67
$RDD
68
$PART
69
$UBQ
70
$NULS
71
$PIVX
72
$MTL
73
$FUN
74
$GTC
75
$LINK
76
$LEND
77
$BLOCK
78
$GNX
79
$PAY
80
$NEO
81
$QTUM
82
$BTC
83
$BTM
84
$REQ
85
$MIOTA
86
$ETN
87
$SNM
88
$XDN
89
$BCN
90
$DBC
91
$ETC
92
$NCASH
93
$CPX
94
$BCD
95
$KNC
96
$FSN
97
$QASH
98
$NXT
99
$LCC
100
$REP
101
$VEN
102
$VTC
103
$DTR
104
$XLM
105
$GNO
106
$BAT
107
$DCR
108
$IGNIS
109
$ZEC
110
$STRAT
111
$ARDR
112
$DRGN
113
$SC
114
$THETA
115
$HT
116
$XZC
117
$MED
118
$ICX


In [14]:
extractor = sa.twitter_setup()
app = Flask(__name__)
@app.route("/", methods = ['POST', 'GET'])
def show_tables():    
    #return render_template('view.html',table=refreshTable.getCoinList().to_html(), title='Table name')
    return render_template('view.html')

@app.route("/refresh_table", methods = ['POST'])
def refresh_table():
    if request.method == 'POST':
        coinTable = refreshTable.getCoinList()
        return render_template('view.html',table=coinTable.to_html(),title='Refreshed table')
    
@app.route("/getTable", methods = ['POST'])
def getTable():
    if request.method == 'POST':
        market = coinmarketcap.Market()
        coinList = pd.DataFrame(market.ticker(limit=300)).set_index("id")
        coinList['percent_change_1h'] = coinList['percent_change_1h'].astype(float)
        coinList = coinList.sort_values('percent_change_1h', axis = 'index', ascending = 0)
        coinList['symbol'] = '$' + coinList['symbol']
        a = ['symbol','percent_change_1h','percent_change_24h','percent_change_7d',
             'price_btc','price_usd','market_cap_usd','24h_volume_usd']
        coinList = coinList[a]
        coinList = coinList.to_json(orient='table')
        #coinList = json.dumps(coinList)
        #print(coinList)
        return(coinList)
    
@app.route("/getTweets")
def getTweets():
    coins = request.args.get('coin')
    try:
        tweets = extractor.search(coins, count = 10)
    except sa.tweepy.TweepError:
        print("Tweepy error!")
        return("Tweepy error!")
        #break
    # create a pandas dataframe as follows:
    tweetData = pd.DataFrame(data=[tweet.created_at for tweet in tweets], columns=['Date'])
    # add relevant data:
    tweetData['Tweets'] = np.array([tweet.text for tweet in tweets])
    tweetData['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
    tweetData['RTs']    = np.array([tweet.retweet_count for tweet in tweets])
    # create a column with the result of the analysis:
    tweetData['SA'] = np.array([ sa.analyze_sentiment(tweet) for tweet in tweetData['Tweets'] ])
    print(coins)
    tweetData = tweetData.to_json(orient='table')
    print(tweetData)
    return(tweetData)
    
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0')
#asdf = show_tables(ethsort)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
192.168.0.109 - - [08/Jun/2018 19:53:10] "GET / HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:53:14] "POST /getTable HTTP/1.1" 200 -


{"schema": {"pandas_version":"0.20.0","primaryKey":["id"],"fields":[{"type":"string","name":"id"},{"type":"string","name":"symbol"},{"type":"number","name":"percent_change_1h"},{"type":"string","name":"percent_change_24h"},{"type":"string","name":"percent_change_7d"},{"type":"string","name":"price_btc"},{"type":"string","name":"price_usd"},{"type":"string","name":"market_cap_usd"},{"type":"string","name":"24h_volume_usd"}]}, "data": [{"id":"tomochain","symbol":"$TOMO","percent_change_1h":8.53,"percent_change_24h":"11.28","percent_change_7d":"-8.71","price_btc":"0.00011209","price_usd":"0.857463","market_cap_usd":"47160465.0","24h_volume_usd":"1733610.0"},{"id":"zebi","symbol":"$ZCO","percent_change_1h":4.88,"percent_change_24h":"1.94","percent_change_7d":"-23.71","price_btc":"0.00001080","price_usd":"0.0826241","market_cap_usd":"38172123.0","24h_volume_usd":"676326.0"},{"id":"ins-ecosystem","symbol":"$INS","percent_change_1h":2.53,"percent_change_24h":"6.72","percent_change_7d":"-4.43"

192.168.0.109 - - [08/Jun/2018 19:53:23] "GET /getTweets HTTP/1.1" 200 -


Tweepy error!


192.168.0.109 - - [08/Jun/2018 19:54:31] "GET / HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:54:31] "GET /static/style.css HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:54:31] "GET /static/js/tables.js HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:54:34] "POST /getTable HTTP/1.1" 200 -


...

Tweepy error!


192.168.0.109 - - [08/Jun/2018 19:56:47] "GET / HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:56:48] "GET /static/style.css HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:56:48] "GET /static/js/tables.js HTTP/1.1" 200 -
192.168.0.109 - - [08/Jun/2018 19:56:51] "POST /getTable HTTP/1.1" 200 -


{"schema": {"pandas_version":"0.20.0","primaryKey":["id"],"fields":[{"type":"string","name":"id"},{"type":"string","name":"symbol"},{"type":"number","name":"percent_change_1h"},{"type":"string","name":"percent_change_24h"},{"type":"string","name":"percent_change_7d"},{"type":"string","name":"price_btc"},{"type":"string","name":"price_usd"},{"type":"string","name":"market_cap_usd"},{"type":"string","name":"24h_volume_usd"}]}, "data": [{"id":"tomochain","symbol":"$TOMO","percent_change_1h":7.16,"percent_change_24h":"11.28","percent_change_7d":"-8.73","price_btc":"0.00011207","price_usd":"0.857326","market_cap_usd":"47152930.0","24h_volume_usd":"1728920.0"},{"id":"zebi","symbol":"$ZCO","percent_change_1h":4.52,"percent_change_24h":"1.94","percent_change_7d":"-23.78","price_btc":"0.00001079","price_usd":"0.0825525","market_cap_usd":"38139044.0","24h_volume_usd":"678630.0"},{"id":"cryptonex","symbol":"$CNX","percent_change_1h":2.54,"percent_change_24h":"-0.13","percent_change_7d":"1.96","pr

192.168.0.109 - - [08/Jun/2018 19:57:00] "GET /getTweets HTTP/1.1" 200 -


Tweepy error!


In [31]:
market = coinmarketcap.Market()
coinList = pd.DataFrame(market.ticker(limit = 180)).set_index("id")
coinList['percent_change_1h'] = coinList['percent_change_1h'].astype(float)
coinList = coinList.sort_values('percent_change_1h', axis = 'index', ascending = 0)
coinList['symbol'] = '$' + coinList['symbol']
a = ['symbol','percent_change_1h','percent_change_24h','percent_change_7d',
     'price_btc','price_usd','24h_volume_usd']
coinList = coinList[a]
coinList = coinList.to_json(orient='table')
#print(coinList)
print (json.dumps(json.loads(coinList), indent=4))

{
    "data": [
        {
            "24h_volume_usd": "1189520.0",
            "id": "zencash",
            "percent_change_1h": 3.63,
            "percent_change_24h": "10.39",
            "percent_change_7d": "38.77",
            "price_btc": "0.00413652",
            "price_usd": "32.6528",
            "symbol": "$ZEN"
        },
        {
            "24h_volume_usd": "53737.8",
            "id": "dynamic-trading-rights",
            "percent_change_1h": 2.85,
            "percent_change_24h": "7.92",
            "percent_change_7d": "55.46",
            "price_btc": "0.00000911",
            "price_usd": "0.071926",
            "symbol": "$DTR"
        },
        {
            "24h_volume_usd": "348419000.0",
            "id": "golem-network-tokens",
            "percent_change_1h": 2.31,
            "percent_change_24h": "64.3",
            "percent_change_7d": "159.25",
            "price_btc": "0.00006145",
            "price_usd": "0.485092",
            "symbol": "$GNT"
   

In [16]:
print(tweetData[1])

KeyError: 1